# Evidencia 1. Actividad Integradora
## Parte 1. Sistemas multiagentes

Ruy Guzmán Camacho | A01639912

Adrián Becerra Meza | A01639813

Santiago González de Cosío Romero | A01640329

#### Librerías


In [69]:
#model design
import agentpy as ap

#visualizations
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import IPython
import random
import time




### Definición de modelo

In [70]:
from select import select
from tracemalloc import start
from turtle import circle


pos = [0, 1]
moves = [(0,-1), (1, 0)]

#Controls

UP = (1,0)
DOWN = (-1,0)
RIGHT = (0,1)
LEFT = (0,-1)
DOWNRIGHT = (-1,1)
DOWNLEFT = (-1,-1)

MOVES = [UP, DOWN, RIGHT, LEFT, DOWNRIGHT, DOWNLEFT]

class StreetModel(ap.Model):

    def makeCircle(self,startX,startY,r):
        map = []
        Epsilon = 2.2
        for y in range(startX-r,startX+r+1):
            for x in range(startY-r,startY+r+1):
                # see if we're close to (x-a)**2 + (y-b)**2 == r**2
                if abs((x-startX)**2 + (y-startY)**2 - r**2) < Epsilon**2:
                    map.append((x,y))
        return map
        
    def setup(self):

        # auxiliar variables for positioning cars
        self.h_count = 0
        self.v_count = 0
        self.flag = 0

        # Create agents (trees)
        n_trees = int(self.p['Tree density'] * (self.p.size**2))
        trees = self.agents = ap.AgentList(self, n_trees)
        self.carros = ap.AgentList(self, self.p['trafico'])

        # Create grid (street)
        self.street = ap.Grid(self, [self.p.size]*2, track_empty=True)
        #self.street.add_agents(trees, random=True, empty=True)
        self.street.add_agents(self.carros,[(2,20)]*self.p['trafico'])

        # Initiate a dynamic variable for all elements
        # Condition 0: trees, 1: car, 2: street 
        # position 0: top, 1: left
        # Draw trees
        
        self.agents.condition = 0
    
        # Draw Street
        #outerPoints = self.makeCircle(int(self.p.size/2),int(self.p.size/2),8)
        #self.outerCrossroad = ap.AgentList(self,len(outerPoints))
       

        innerPoints = self.makeCircle(int(self.p.size/2),int(self.p.size/2), 7)
        self.road = innerPoints
        self.innerCrossroad = ap.AgentList(self,len(innerPoints)*2)


        #self.street.add_agents(self.outerCrossroad,outerPoints)
        self.street.add_agents(self.innerCrossroad,innerPoints)
        
        #self.outerCrossroad.condition = 2
        self.innerCrossroad.condition = 0
        
        #street1 = self.street.agents[self.circlePoints]
        #street2 = self.street.agents[int(self.p.size/8)*3:int(self.p.size/8)*5, int(self.p.size/8)*3:int(self.p.size/8)*5]
        #street1.condition = 2
        #street2.condition = 2

        # draw cars
        self.carros.condition = 1
        # position cars
        """for carro in self.carros:
            carro.position = random.choice(pos)
            if carro.position == 0:
                self.h_count += 1
                #horizontal
                self.street.move_to(carro, (int(self.p.size*0.5),2))
            else:
                #vertical
                self.v_count += 1
                self.street.move_to(carro, (2,int(self.p.size*0.5)))"""



    def step(self):

        # Select cars
        carros = self.carros
        listcars = list(carros)
        road = self.road
        
        
        #tree = self.agents.select(self.agents.condition == 0)

        #para que siga la ruta
        

        # move cars depending on position
        for carro in carros:
            #bajara hasta que se tope con la glorieta
            positions = self.street.positions
            print(positions[listcars[0]])
            
            neighbors = self.street.neighbors(carro)
            listNeighbor = list(neighbors)
            
            if listNeighbor:
                for neighbor in listNeighbor:
                    if neighbor.condition == 0:
                        print(neighbor)
                        self.street.move_to(carro, positions[neighbor])
                        neighbor.condition = 1
                    '''else:
                        neighbor.condition = 0
                        '''
            else:
                self.street.move_by(carro, (1,0))
                
            """if carro.position == 1:
                self.street.move_by(carro, (random.choice(pos),0))
            else:
                self.street.move_by(carro, (0,random.choice(pos)))"""




        burning_trees = self.agents.select(self.agents.condition == 0)
        if len(burning_trees) == 0:
            self.stop()


    def end(self):
        
        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees',
                    burned_trees / len(self.agents))
                    

### Parametros

In [71]:
# Define parameters

parameters = {
    'Tree density': 1, # Percentage of grid covered by trees
    'size': 40,
    'n': 20, # Height and length of the grid
    'steps': 50,
    'trafico': 1
}


### Renderizado

In [72]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#808080', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
circle1 = plt.Circle((20,20), 10, color='r')

ax.add_patch(circle1)
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=50))

(2, 20)
(3, 20)
(4, 20)
(5, 20)
(6, 20)
(7, 20)
(8, 20)
(9, 20)
(10, 20)
(11, 20)
(12, 20)
Agent (Obj 1620)
Agent (Obj 1622)
Agent (Obj 1624)
(13, 21)
Agent (Obj 1626)
(13, 22)
Agent (Obj 1628)
(14, 23)
Agent (Obj 1630)
(14, 24)
Agent (Obj 1632)
(15, 25)
Agent (Obj 1634)
(16, 26)
Agent (Obj 1635)
(17, 26)
Agent (Obj 1638)
(18, 27)
Agent (Obj 1639)
(19, 27)
Agent (Obj 1640)
(20, 27)
Agent (Obj 1641)
(21, 27)
Agent (Obj 1642)
(22, 27)
Agent (Obj 1636)
(23, 26)
Agent (Obj 1637)
(24, 26)
Agent (Obj 1633)
(25, 25)
Agent (Obj 1631)
(26, 24)
Agent (Obj 1629)
(26, 23)
Agent (Obj 1627)
(27, 22)
Agent (Obj 1625)
(27, 21)
Agent (Obj 1623)
(27, 20)
Agent (Obj 1621)
(27, 19)
Agent (Obj 1619)
(27, 18)
Agent (Obj 1617)
(26, 17)
Agent (Obj 1615)
(26, 16)
Agent (Obj 1613)
(25, 15)
Agent (Obj 1611)
(24, 14)
Agent (Obj 1610)
(23, 14)
Agent (Obj 1607)
(22, 13)
Agent (Obj 1606)
(21, 13)
Agent (Obj 1605)
(20, 13)
Agent (Obj 1604)
(19, 13)
Agent (Obj 1603)
(18, 13)
Agent (Obj 1609)
(17, 14)
Agent (Obj 1608)
